In [1]:
import numpy as np
import pandas as pd
import os
import random

#Data Source
import yfinance as yf

#Display
import ipywidgets as widgets
from IPython.display import Image
import plotly.graph_objs as go

In [2]:
out_pl = widgets.Output()

In [3]:
def analyze(change):
    
    out_pl.clear_output()
    
    if ticker_box.value == "":
        return
    
    #Set parameters
    ticker = ticker_box.value
    period = '1d'
    interval = '1m'
    mov_avg_window=21
    
    #Download data
    data = yf.download(tickers=ticker, period=period, interval=interval, 
                           progress=False, show_errors=False)
    if data.empty:
        with out_pl: print(f"{ticker_box.value} data failed to download, please try again.")
        return
    
    data.index = data.index.tz_convert('America/Los_Angeles')
    
    #Calculate Moving Average and Ballinger Bands
    mov_avg_title = str(mov_avg_window) + 'd Moving Avg'
    data[mov_avg_title] = data['Close'].rolling(window=mov_avg_window).mean()
    data['Upper Band'] = data[mov_avg_title] + 1.96*data['Close'].rolling(window=mov_avg_window).std()
    data['Lower Band'] = data[mov_avg_title] - 1.96*data['Close'].rolling(window=mov_avg_window).std()
    
    #declare figure
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=data.index, y= data[mov_avg_title],
                             line=dict(color='blue', width=.7), 
                             name = mov_avg_title))
    fig.add_trace(go.Scatter(x=data.index, y= data['Upper Band'],
                             line=dict(color='red', width=1.5), 
                             name = 'Upper Band (Sell)'))
    fig.add_trace(go.Scatter(x=data.index, y= data['Lower Band'],
                             line=dict(color='green', width=1.5), 
                             name = 'Lower Band (Buy)'))

    #Candlestick
    fig.add_trace(go.Candlestick(x=data.index,
                    open=data['Open'],
                    high=data['High'],
                    low=data['Low'],
                    close=data['Close'], name = 'market data'))

    # Add titles
    fig.update_layout(
        title=ticker.upper(),
        yaxis_title='Stock Price')

    # X-Axes
    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=15, label="15m", step="minute", stepmode="backward"),
                dict(count=45, label="45m", step="minute", stepmode="backward"),
                dict(count=3, label="3h", step="hour", stepmode="backward"),
                dict(label=period, step="all")
            ])
        )
    )

    #Select random buy gif
    buy_path = 'assets/buy/'
    files=os.listdir(buy_path)
    
    #Display
    with out_pl: 
        display(Image(filename=buy_path + random.choice(files)))
        fig.show()

In [4]:
ticker_box = widgets.Text(
    value='',
    placeholder='e.g. AAPL, TQQQ',
    disabled=False
)

In [5]:
analyze_button = widgets.Button(
    description='Analyze',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
)

analyze_button.on_click(analyze)

In [6]:
widgets.VBox([widgets.Label('Enter Ticker'), 
    ticker_box, 
    analyze_button,
    out_pl])